In [1]:
const liblapack = Base.liblapack_name
import Base.LinAlg.BlasInt, Base.LinAlg.BlasReal, Base.blas_vendor, Base.LinAlg.Eigen

if blas_vendor() == :openblas64
    macro blasfunc(x)
        return Expr(:quote, symbol(x, "64_"))
    end
end

type LAPACKException <: Exception
    info::BlasInt
end

function chklapackerror(ret::BlasInt)
    if ret == 0
        return
    elseif ret < 0
        throw(ArgumentError("invalid argument #$(-ret) to LAPACK call"))
    else # ret > 0
        throw(LAPACKException(ret))
    end
end

chklapackerror (generic function with 1 method)

In [2]:
stegr = :dstegr_
elty = :Float64

@eval begin
    function bar!(jobz::Char, range::Char, dv::Vector{$elty}, ev::Vector{$elty}, vl::Real, vu::Real, il::Integer, iu::Integer)
        n = length(dv)
        if length(ev) != n - 1
            throw(DimensionMismatch("ev has length $(length(ev)) but needs one less than dv's length, $n)"))
        end
        eev = [ev; zero($elty)]
        abstol = Array($elty, 1)
        m = Array(BlasInt, 1)
        w = similar(dv, $elty, n)
        ldz = jobz == 'N' ? 1 : n
        Z = similar(dv, $elty, ldz, n)
        isuppz = similar(dv, BlasInt, 2n)
        work = Array($elty, 1)
        lwork = BlasInt(-1)
        iwork = Array(BlasInt, 1)
        liwork = BlasInt(-1)
        info = Ref{BlasInt}()
        for i = 1:2
            ccall((@blasfunc($stegr), liblapack), Void,
                (Ptr{UInt8}, Ptr{UInt8}, Ptr{BlasInt}, Ptr{$elty},
                Ptr{$elty}, Ptr{$elty}, Ptr{$elty}, Ptr{BlasInt},
                Ptr{BlasInt}, Ptr{$elty}, Ptr{BlasInt}, Ptr{$elty},
                Ptr{$elty}, Ptr{BlasInt}, Ptr{BlasInt}, Ptr{$elty},
                Ptr{BlasInt}, Ptr{BlasInt}, Ptr{BlasInt}, Ptr{BlasInt}),
                &jobz, &range, &n, dv,
                eev, &vl, &vu, &il,
                &iu, abstol, m, w,
                Z, &ldz, isuppz, work,
                &lwork, iwork, &liwork, info)
            chklapackerror(info[])
            if i == 1
                lwork = BlasInt(work[1])
                work = Array($elty, lwork)
                liwork = iwork[1]
                iwork = Array(BlasInt, liwork)
            end
        end
        w[1:m[1]], Z[:,1:m[1]]
    end
end
bar!(jobz::Char, dv::Vector, ev::Vector) = bar!(jobz, 'A', dv, ev, 0.0, 0.0, 0, 0)

function my_eig{T<:Float64}(A::SymTridiagonal{T})
    F = Eigen(bar!('V', A.dv, A.ev)...)
    F.values, F.vectors
end

my_eig (generic function with 1 method)

In [3]:
include("utils.jl")
using Utils

In [4]:
A, M = Utils.matrices(1000, :SymTridiagonal);
@time w, _ = my_eig(A);
A, M = Utils.matrices(1000, :SymTridiagonal);
@time w0, _ = eig(A);
norm(w-w0)

  0.247240 seconds (271.64 k allocations: 28.724 MB, 2.58% gc time)
  

0.0

0.108258 seconds (50.10 k allocations: 17.790 MB, 2.70% gc time)


In [4]:
stegr = :dstevr_
elty = :Float64

@eval begin
    function foo!(jobz::Char, range::Char, dv::Vector{$elty}, ev::Vector{$elty}, vl::Real, vu::Real, il::Integer, iu::Integer)
        n = length(dv)
        if length(ev) != n - 1
            throw(DimensionMismatch("ev has length $(length(ev)) but needs one less than dv's length, $n)"))
        end
        eev = [ev; zero($elty)]
        abstol = Array($elty, 1)
        m = Array(BlasInt, 1)
        w = similar(dv, $elty, n)
        ldz = jobz == 'N' ? 1 : n
        Z = similar(dv, $elty, ldz, n)
        isuppz = similar(dv, BlasInt, 2n)
        work = Array($elty, 1)
        lwork = BlasInt(-1)
        iwork = Array(BlasInt, 1)
        liwork = BlasInt(-1)
        info = Ref{BlasInt}()
        for i = 1:2
            ccall((@blasfunc($stegr), liblapack), Void,
                (Ptr{UInt8}, Ptr{UInt8}, Ptr{BlasInt}, Ptr{$elty},
                Ptr{$elty}, Ptr{$elty}, Ptr{$elty}, Ptr{BlasInt},
                Ptr{BlasInt}, Ptr{$elty}, Ptr{BlasInt}, Ptr{$elty},
                Ptr{$elty}, Ptr{BlasInt}, Ptr{BlasInt}, Ptr{$elty},
                Ptr{BlasInt}, Ptr{BlasInt}, Ptr{BlasInt}, Ptr{BlasInt}),
                &jobz, &range, &n, dv,
                eev, &vl, &vu, &il,
                &iu, abstol, m, w,
                Z, &ldz, isuppz, work,
                &lwork, iwork, &liwork, info)
            chklapackerror(info[])
            if i == 1
                lwork = BlasInt(work[1])
                work = Array($elty, lwork)
                liwork = iwork[1]
                iwork = Array(BlasInt, liwork)
            end
        end
        w[1:m[1]], Z[:,1:m[1]]
    end
end
foo!(jobz::Char, dv::Vector, ev::Vector) = foo!(jobz, 'A', dv, ev, 0.0, 0.0, 0, 0)

function my_eig_foo{T<:Float64}(A::SymTridiagonal{T})
    F = Eigen(foo!('V', A.dv, A.ev)...)
    F.values, F.vectors
end

my_eig_foo (generic function with 1 method)

In [20]:
m = 2^3

A, M = Utils.matrices(m, :SymTridiagonal);
@time w, v = my_eig_foo(A);
A, M = Utils.matrices(m, :SymTridiagonal);
@time w0, v0 = eig(A);

println("$(norm(w-w0, Inf)) $(norm(v-v0, Inf))")


LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Utils.cols{T<:Real}}, ::SymTridiagonal{Float64})
This may have arisen from a call to the constructor Utils.cols{T<:Real}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert{T}(::Type{T}, !Matched::T)
  Utils.cols{T<:Real}(!Matched::Array{T<:Real,2}, !Matched::OrdinalRange{Int64,Int64})
  ...
while loading In[20], in expression starting on line 10

0.000115 seconds (57 allocations: 6.625 KB)
  0.000442 seconds (62 allocations: 6.844 KB)
0.0 0.0


In [16]:
stegr = :dstevx_
elty = :Float64

@eval begin
    function cux!(jobz::Char, range::Char, dv::Vector{$elty}, ev::Vector{$elty}, vl::Real, vu::Real, il::Integer, iu::Integer)
        n = length(dv)
        if length(ev) != n - 1
            throw(DimensionMismatch("ev has length $(length(ev)) but needs one less than dv's length, $n)"))
        end
        eev = [ev; zero($elty)]
        abstol = Array($elty, 1)
        m = Array(BlasInt, 1)
        w = similar(dv, $elty, n)
        ldz = jobz == 'N' ? 1 : n
        Z = similar(dv, $elty, ldz, n)
        ifail = similar(dv, BlasInt, n)
        work = Array($elty, 5n)
        iwork = Array(BlasInt, 5n)
        info = Ref{BlasInt}()
        
        ccall((@blasfunc($stegr), liblapack), Void,
            (Ptr{UInt8}, Ptr{UInt8}, Ptr{BlasInt},
             Ptr{$elty}, Ptr{$elty}, 
             Ptr{$elty}, Ptr{$elty}, Ptr{BlasInt}, Ptr{BlasInt}, 
             Ptr{$elty}, Ptr{BlasInt}, 
             Ptr{$elty}, Ptr{$elty}, 
             Ptr{BlasInt}, Ptr{$elty}, Ptr{BlasInt},
             Ptr{BlasInt}, Ptr{BlasInt}),
            &jobz, &range, &n, 
            dv, eev,
            &vl, &vu, &il, &iu,
            abstol, m,
            w, Z, 
            &ldz, work, iwork, 
            ifail, info)
        chklapackerror(info[])
        
        w[1:m[1]], Z[:,1:m[1]]
    end
end
cux!(jobz::Char, dv::Vector, ev::Vector) = cux!(jobz, 'A', dv, ev, 0.0, 0.0, 0, 0)

function my_eig_cux{T<:Float64}(A::SymTridiagonal{T})
    F = Eigen(cux!('V', A.dv, A.ev)...)
    F.values, F.vectors
end

my_eig_cux (generic function with 1 method)

  0.023445 seconds (24.53 k allocations: 1.142 MB)
  0.002274 seconds (57 allocations: 28.516 KB)
5.88418203051333e-14 5.294946348576806
95.89402073482518
0.0
0.0
95.98695561217113
96.0733999801841
96.15337147968046
96.2268863417538
96.29395942381556
96.35460423302878
96.40883294432506
96.45665641522565
96.498084198358
96.53312455209188
96.56178444952226
96.58406958593172
96.59998438481568
96.60953200252418
96.61271433155548
96.60953200252416
96.59998438481568
96.58406958593172
96.56178444952228
96.53312455209188
96.498084198358
96.45665641522565
96.40883294432506
96.3546042330288
96.29395942381558
96.22688634175381
96.15337147968046
96.0733999801841
95.98695561217112
95.89402073482518


In [8]:
stegr = :dpteqr_
elty = :Float64

@blasfunc dpteqr_



const liblapack = Base.liblapack_name
import Base.LinAlg.BlasInt, Base.LinAlg.Eigen


"""
Computes all eigenvalues and, optionally, eigenvectors of a symmetric positive definite tridiagonal matrix.
"""
function spd3_eigfact!(compz::Char, dv::Vector{Float64}, ev::Vector{Float64})
    n = length(dv)                    # N
    if length(ev) != n - 1
        throw(DimensionMismatch("ev has length $(length(ev)) but needs one less than dv's length, $n)"))
    end

    d = dv                            # D
    e = [ev; zero(Float64)]           # E
    ldz = compz == 'N' ? 1 : n        # LDZ
    Z = similar(d, Float64, ldz, n)   # Z
    work = zeros(Float64, 4n)         # WORK
    info = Ref{BlasInt}()             # Info

    ccall((:dpteqr_64_, liblapack), Void,
          (Ptr{UInt8},    Ptr{BlasInt}, Ptr{Float64}, Ptr{Float64},
           Ptr{Float64}, Ptr{BlasInt}, Ptr{Float64}, Ptr{BlasInt}), 
           &compz,        &n,           d,             e,
           Z,             &ldz,         work,       info)

    # @assert info[] == 0
    println("....", info[])

    d[1:n], Z[:,1:n]
end


# FOO!(jobz::Char, dv::Vector, ev::Vector) = FOO!(jobz, 'I', dv, ev, 0.0, 0.0, 0, 0)

#function my_eig{T<:Float64}(A::SymTridiagonal{T})
#    F = Eigen(bar!('V', A.dv, A.ev)...)
#    F.values, F.vectors
#end


spd3_eigfact! (generic function with 1 method)

In [9]:
include("utils.jl")
using Utils
A, M = Utils.matrices(10, :SymTridiagonal);
A;

In [5]:
w, _ = eig(A);
w

11-element Array{Float64,1}:
  0.97887
  1.0    
  1.0    
  3.81966
  8.24429
 13.8197 
 20.0    
 26.1803 
 31.7557 
 36.1803 
 39.0211 

In [10]:
d, e = A.dv, A.ev

([1.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,1.0],[0.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,0.0])

In [11]:
spd3_eigfact!('V', d, e)

....0


([39.02113032590308,36.18033988749899,31.755705045849503,26.180339887498963,20.00000000000001,13.819660112501042,8.244294954150536,3.8196601125010456,1.0,1.0,0.9788696740969282],
11x11 Array{Float64,2}:
 -1.95354e-311   5.89836e-310  3.57088e-310  …  0.0           1.52472e-309
 -3.29547e-310   5.89836e-310  6.67206e-310     0.0           1.21462e-309
 -2.16969e-310   4.77184e-310  1.89972e-310     6.93415e-310  1.10199e-309
  3.39786e-311   1.82264e-310  2.85861e-310     6.93373e-310  1.35285e-309
 -2.61032e-310  -1.79697e-319  4.68069e-310     0.0           1.64778e-309
 -2.6093e-310    5.86609e-318  4.68125e-310  …  0.0           1.64776e-309
  3.38956e-311   1.82275e-310  2.85794e-310     6.93415e-310  1.35285e-309
 -2.16901e-310   4.77191e-310  1.90033e-310     6.93373e-310  1.10197e-309
 -3.29621e-310   5.89836e-310  6.67174e-310     6.93415e-310  1.21464e-309
 -7.86666e-311   2.94909e-310  7.63034e-310     6.93373e-310  1.4655e-309 
  3.38956e-311  -1.82275e-310  2.85794e-310  … 

In [105]:
d

11-element Array{Float64,1}:
 39.0211 
 36.1803 
 31.7557 
 26.1803 
 20.0    
 13.8197 
  8.24429
  3.81966
  1.0    
  1.0    
  0.97887

In [7]:
e

10-element Array{Float64,1}:
   0.0
 -10.0
 -10.0
 -10.0
 -10.0
 -10.0
 -10.0
 -10.0
 -10.0
   0.0

In [107]:
promote_type(Int32, Float64)

LoadError: LoadError: no promotion exists for Int32 and Union{Float32,Float64}
while loading In[107], in expression starting on line 1

In [109]:
?copy_oftype

search:

Couldn't find copy_oftype
Perhaps you meant code_typed


LoadError: LoadError: "copy_oftype" is not defined in module Main
while loading In[109], in expression starting on line 119